In [1]:
from IPython.display import Markdown, display

md = """
# LithoBench 데이터 폴더 의미

| 폴더명 | 의미(Meaning) | 공정 단계 | 설명 |
|---|---|---|---|
| target | 목표 패턴 | 입력 (Input) | 웨이퍼에 최종적으로 찍혀야 하는 이상적인 회로 도면입니다. (설계자 의도) |
| pixelILT | 픽셀 기반 마스크 | 최적화 (Opt 1) | target을 구현하기 위해 픽셀 단위로 최적화된 마스크 패턴입니다. (모자이크 형태) |
| levelsetILT | 곡선 기반 마스크 | 최적화 (Opt 2) | 레벨셋 알고리즘으로 생성된 부드러운 곡선(Curvilinear) 형태의 마스크입니다. (최신 공정 트렌드) |
| litho | 광학 이미지 | 시뮬레이션 (Sim) | 마스크를 통과한 빛이 감광액에 도달했을 때의 에너지 분포(Aerial Image)입니다. |
| resist | 감광액 패턴 | 결과 (Output) | litho 이미지에 임계값(Threshold)을 적용해 실제로 남게 된 패턴입니다. |
| printed | 최종 식각 패턴 | 최종 결과 | 현상(Develop) 및 식각(Etching) 후의 최종 형상입니다. (resist와 거의 유사할 수 있음) |
| gds / glp | 벡터/메타데이터 | 참조 (Ref) | 이미지의 원본 벡터 파일(gds)과 텍스트 파라미터(glp)입니다. |
"""

display(Markdown(md))



# LithoBench 데이터 폴더 의미

| 폴더명 | 의미(Meaning) | 공정 단계 | 설명 |
|---|---|---|---|
| target | 목표 패턴 | 입력 (Input) | 웨이퍼에 최종적으로 찍혀야 하는 이상적인 회로 도면입니다. (설계자 의도) |
| pixelILT | 픽셀 기반 마스크 | 최적화 (Opt 1) | target을 구현하기 위해 픽셀 단위로 최적화된 마스크 패턴입니다. (모자이크 형태) |
| levelsetILT | 곡선 기반 마스크 | 최적화 (Opt 2) | 레벨셋 알고리즘으로 생성된 부드러운 곡선(Curvilinear) 형태의 마스크입니다. (최신 공정 트렌드) |
| litho | 광학 이미지 | 시뮬레이션 (Sim) | 마스크를 통과한 빛이 감광액에 도달했을 때의 에너지 분포(Aerial Image)입니다. |
| resist | 감광액 패턴 | 결과 (Output) | litho 이미지에 임계값(Threshold)을 적용해 실제로 남게 된 패턴입니다. |
| printed | 최종 식각 패턴 | 최종 결과 | 현상(Develop) 및 식각(Etching) 후의 최종 형상입니다. (resist와 거의 유사할 수 있음) |
| gds / glp | 벡터/메타데이터 | 참조 (Ref) | 이미지의 원본 벡터 파일(gds)과 텍스트 파라미터(glp)입니다. |


In [2]:
from pathlib import Path
import re
import csv

# 데이터셋 루트
root = Path(r"C:\Users\asap0\OneDrive\바탕 화면\yonsei\26-1 DSL\eda\SemiConductor_EDA_2602\LithoBench\0_Datasets\lithodata")

# 분석 대상 세트 선택 (예: "ViaSet", "MetalSet", "StdContact", "StdContactTest", "StdMetal")
dataset_name = "ViaSet"

dataset_dir = root / dataset_name
if not dataset_dir.exists():
    raise FileNotFoundError(dataset_dir)

# 비교할 폴더들
folders = ["target", "pixelILT", "levelsetILT", "litho", "resist", "printed", "glp", "gds"]

# 각 폴더에서 stem(확장자 제외) 수집
stems_by_folder = {}
for f in folders:
    d = dataset_dir / f
    if not d.exists():
        stems_by_folder[f] = set()
        continue
    stems_by_folder[f] = {p.stem for p in d.iterdir() if p.is_file()}

# 전체 stem 집합
all_stems = set().union(*stems_by_folder.values())

# glp 간단 파싱: key=value 형태만 추출
kv_re = re.compile(r"^\s*([A-Za-z0-9_\-]+)\s*=\s*(.+)\s*$")

def parse_glp(path: Path):
    kv = {}
    try:
        with path.open("r", encoding="utf-8", errors="ignore") as f:
            for line in f:
                m = kv_re.match(line)
                if m:
                    kv[m.group(1)] = m.group(2)
    except Exception:
        return {}
    return kv

# CSV 저장용 rows
rows = []

for stem in sorted(all_stems):
    row = {"dataset": dataset_name, "stem": stem}
    for f in folders:
        row[f"has_{f}"] = stem in stems_by_folder[f]

    # glp 메타데이터 요약
    glp_path = dataset_dir / "glp" / f"{stem}.glp"
    if glp_path.exists():
        kv = parse_glp(glp_path)
        row["glp_keys"] = ";".join(sorted(kv.keys())[:10])
        row["glp_kv_count"] = len(kv)
    else:
        row["glp_keys"] = ""
        row["glp_kv_count"] = 0

    rows.append(row)

# 요약 출력
print(f"Dataset: {dataset_name}")
for f in folders:
    print(f"- {f}: {len(stems_by_folder[f])} files")

# 누락 확인
print("\n[Missing report]")
for f in folders:
    missing = [s for s in all_stems if s not in stems_by_folder[f]]
    print(f"- missing in {f}: {len(missing)}")

# CSV 저장
out_csv = dataset_dir / f"{dataset_name}_file_match_report.csv"
with out_csv.open("w", newline="", encoding="utf-8") as f:
    fieldnames = ["dataset", "stem"] + [f"has_{f}" for f in folders] + ["glp_keys", "glp_kv_count"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

print(f"\nCSV 저장 완료: {out_csv}")


Dataset: ViaSet
- target: 116414 files
- pixelILT: 116405 files
- levelsetILT: 111958 files
- litho: 116405 files
- resist: 116405 files
- printed: 116405 files
- glp: 116414 files
- gds: 0 files

[Missing report]
- missing in target: 10
- missing in pixelILT: 19
- missing in levelsetILT: 4466
- missing in litho: 19
- missing in resist: 19
- missing in printed: 19
- missing in glp: 10
- missing in gds: 116424

CSV 저장 완료: C:\Users\asap0\OneDrive\바탕 화면\yonsei\26-1 DSL\eda\SemiConductor_EDA_2602\LithoBench\0_Datasets\lithodata\ViaSet\ViaSet_file_match_report.csv


In [4]:
from pathlib import Path
import csv
import re
import numpy as np
from PIL import Image

# 데이터셋 루트 및 대상 세트
root = Path(r"C:\Users\asap0\OneDrive\바탕 화면\yonsei\26-1 DSL\eda\SemiConductor_EDA_2602\LithoBench\0_Datasets\lithodata")
dataset_name = "ViaSet"
dataset_dir = root / dataset_name

# 비교할 이미지 폴더 쌍 (A vs B)
compare_pairs = [
    ("target", "printed"),
    ("target", "resist"),
    ("target", "litho"),
]

# 처리 속도 제어
max_files = 2000  # None이면 전체 수행
print_every = 200

# glp 파싱 (key=value)
kv_re = re.compile(r"^\s*([A-Za-z0-9_\-]+)\s*=\s*(.+)\s*$")

def parse_glp(path: Path):
    kv = {}
    try:
        with path.open("r", encoding="utf-8", errors="ignore") as f:
            for line in f:
                m = kv_re.match(line)
                if m:
                    kv[m.group(1)] = m.group(2)
    except Exception:
        return {}
    return kv

# 이미지 로드 (그레이스케일)

def load_gray(path: Path):
    with Image.open(path) as img:
        return np.array(img.convert("L"), dtype=np.float32)

# 지표 계산

def mse(a, b):
    return float(np.mean((a - b) ** 2))


def mae(a, b):
    return float(np.mean(np.abs(a - b)))


def psnr(a, b):
    m = mse(a, b)
    if m == 0:
        return float("inf")
    return float(20 * np.log10(255.0) - 10 * np.log10(m))


# 파일 stem 기준으로 매칭: 비교쌍의 교집합만 사용
folders = sorted({f for pair in compare_pairs for f in pair})
stems_by_folder = {}
for f in folders:
    d = dataset_dir / f
    stems_by_folder[f] = {p.stem for p in d.iterdir() if p.is_file()} if d.exists() else set()

if stems_by_folder:
    common_stems = set.intersection(*stems_by_folder.values())
else:
    common_stems = set()

stems = sorted(common_stems)
if max_files is not None:
    stems = stems[:max_files]

# CSV 저장 (스트리밍)
out_csv = dataset_dir / f"{dataset_name}_pixel_compare.csv"
fieldnames = ["dataset", "stem", "glp_kv_count", "glp_keys"]
for a_name, b_name in compare_pairs:
    key_prefix = f"{a_name}_vs_{b_name}"
    fieldnames += [
        f"{key_prefix}_status",
        f"{key_prefix}_mse",
        f"{key_prefix}_mae",
        f"{key_prefix}_psnr",
    ]

with out_csv.open("w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    for i, stem in enumerate(stems, start=1):
        row = {"dataset": dataset_name, "stem": stem}

        # glp 정보 요약
        glp_path = dataset_dir / "glp" / f"{stem}.glp"
        if glp_path.exists():
            kv = parse_glp(glp_path)
            row["glp_kv_count"] = len(kv)
            row["glp_keys"] = ";".join(sorted(kv.keys())[:10])
        else:
            row["glp_kv_count"] = 0
            row["glp_keys"] = ""

        # 이미지 비교
        for a_name, b_name in compare_pairs:
            a_path = dataset_dir / a_name / f"{stem}.png"
            b_path = dataset_dir / b_name / f"{stem}.png"
            key_prefix = f"{a_name}_vs_{b_name}"
            if a_path.exists() and b_path.exists():
                a = load_gray(a_path)
                b = load_gray(b_path)
                if a.shape != b.shape:
                    row[f"{key_prefix}_status"] = "shape_mismatch"
                    row[f"{key_prefix}_mse"] = ""
                    row[f"{key_prefix}_mae"] = ""
                    row[f"{key_prefix}_psnr"] = ""
                else:
                    row[f"{key_prefix}_status"] = "ok"
                    row[f"{key_prefix}_mse"] = mse(a, b)
                    row[f"{key_prefix}_mae"] = mae(a, b)
                    row[f"{key_prefix}_psnr"] = psnr(a, b)
            else:
                row[f"{key_prefix}_status"] = "missing"
                row[f"{key_prefix}_mse"] = ""
                row[f"{key_prefix}_mae"] = ""
                row[f"{key_prefix}_psnr"] = ""

        writer.writerow(row)

        if i % print_every == 0:
            print(f"Processed {i}/{len(stems)}")

print(f"\nCSV 저장 완료: {out_csv}")
print(f"총 처리 파일 수: {len(stems)}")


Processed 200/2000
Processed 400/2000
Processed 600/2000
Processed 800/2000
Processed 1000/2000
Processed 1200/2000
Processed 1400/2000
Processed 1600/2000
Processed 1800/2000
Processed 2000/2000

CSV 저장 완료: C:\Users\asap0\OneDrive\바탕 화면\yonsei\26-1 DSL\eda\SemiConductor_EDA_2602\LithoBench\0_Datasets\lithodata\ViaSet\ViaSet_pixel_compare.csv
총 처리 파일 수: 2000
